# 라이브러리

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(transform_output='pandas')

# 데이터 뽑아오는 함수

In [2]:
# 데이터 병합 및 year 열 추가 함수 정의의: SQL
def get_data_from_db(query):

    # mysql 접속 정보 정의
    username = "admin"
    password = "admin1234"
    host = "hk-toss-middle-project.cjkcuqkegqpx.eu-north-1.rds.amazonaws.com"
    database_name = "raw_data"

    # 데이터베이스 연결 문자열, connection string
    db_connection_str = f'mysql+pymysql://{username}:{password}@{host}/{database_name}'

    # 데이터 베이스 connector 얻어내기
    db_connection = create_engine(db_connection_str)
    db_connection

    df = pd.read_sql(query, con=db_connection)
    return df

# Transformer 정의

## 문자열 앞뒤 공백 제거 Transformer

In [3]:
# 앞뒤공백 제거 transformer
class StripTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns:list):
        # 입력한 columns가 문자열이면 리스트로 변환
        if isinstance(columns, str):
            self.columns = [columns]
        else:
            self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # 선택한 열 중에 데이터프레임에 실제로 존재하는 열만 선택
        columns_y = [col for col in self.columns if col in X.columns]

        # 선택한 열의 값들의 앞뒤 공백을 제거
        for col in columns_y:

            # 문자열 형식인지 확인
            if X[col].dtype == 'object':
                try:
                    X[col] = X[col].str.strip()
                except:
                    pass

        return X

## 열 탈락 / 열 이름 변경 Transformer

In [4]:
class RenameDropColumnsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, rename_map:dict = {}, drop_columns:list = []):
        self.rename_map = rename_map
        # drop_columns가 문자열이면 리스트로 변환
        if isinstance(drop_columns, str):
            self.drop_columns = [drop_columns]
        else:
            self.drop_columns = drop_columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # 실제로 데이터프레임에 존재하는 열만 drop
        self.drop_columns = [col for col in self.drop_columns if col in X.columns]
        # drop_columns가 존재하면 drop
        if self.drop_columns:
            X.drop(columns=self.drop_columns, inplace=True)

        # 실제로 데이터프레임에 존재하는 열만 선택
        rename_map = {key: value for key, value in self.rename_map.items() if key in X.columns}
        # rename_map이 존재하면 rename
        if self.rename_map:
            X.rename(columns=self.rename_map, inplace=True)
            
        return X

## NaN 및 값 대치 처리

In [5]:
class ValueImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns, missing_value, fill_value):
        if isinstance(columns, str):
            self.columns = [columns]
        else: 
            self.columns = columns
        self.missing_value = missing_value
        self.fill_value = fill_value
        
        self.imputer = SimpleImputer(missing_values=self.missing_value, strategy='constant', fill_value=self.fill_value)

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()        
        X_copy[self.columns] = self.imputer.fit_transform(X_copy[self.columns])
        return X_copy

## ManyHotEncoding을 위한 리스트 열 만들어주기

In [6]:
class ColumnsWithList(BaseEstimator, TransformerMixin):
    def __init__(self, from_columns, to_column):
        self.from_columns = from_columns
        self.to_column= to_column

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.to_column] = X[self.from_columns].apply(lambda row: ','.join([str(int(x)) for x in [row[col] for col in self.from_columns]]), axis=1)
        X.drop(columns=self.from_columns, inplace=True)
        return X

## ManyHotEncoding

In [7]:
class ManyHotEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, columns, prefix = ""):
        self.columns = columns
        self.prefix = prefix
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        # MultiLabelBinarizer를 학습합니다.
        # self.mlb.fit(X[self.columns])
        # return self
        return self

    def transform(self, X):
        # MultiLabelBinarizer를 사용하여 데이터를 변환합니다.
        encoded = self.mlb.fit_transform(X[self.columns])
        encoded = encoded[:, 1:]

        # 숫자 클래스와 name_map을 사용하여 열 이름 생성
        mapped_classes = [f"{self.prefix}_{cls}" for cls in self.mlb.classes_[1:]]

        # 변환된 데이터를 DataFrame으로 반환
        encoded_df = pd.DataFrame(encoded, columns=mapped_classes)
        X.drop(columns=self.columns, inplace=True)
        data = pd.concat([encoded_df, X], axis=1)
        return data

# HH Table

In [8]:
columns_hh = "HHID, CUSTM_BENF1_D, CUSTM_BENF2_D, CUSTM_BENF3_D, CUSTM_BENF4_D, H_INC1, H_INC2, H_INC3, H_INC4, H_INC5, H_INC6, H_INC7, H_INC_TOT, H_INC_MON, OTC_MED, OTC_BUY1, OTC_BUY2, OTC_BUY3, HLT_SUP1, HLT_SUP2, MED_SUP, H_OOP, HEXP2, HEXP3"

query_hh = f"""
SELECT 2019 AS D_YEAR, {columns_hh} from raw_a_hh
UNION ALL
SELECT 2020 AS D_YEAR, {columns_hh} from raw_b_hh
UNION ALL
SELECT 2021 AS D_YEAR, {columns_hh} from raw_c_hh"""

hh = get_data_from_db(query_hh)
hh.head()

,D_YEAR,HHID,CUSTM_BENF1_D,CUSTM_BENF2_D,CUSTM_BENF3_D,CUSTM_BENF4_D,H_INC1,H_INC2,H_INC3,H_INC4,...,OTC_MED,OTC_BUY1,OTC_BUY2,OTC_BUY3,HLT_SUP1,HLT_SUP2,MED_SUP,H_OOP,HEXP2,HEXP3
0,2019,112001011.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,618.0,...,61000.0,1.0,3.0,6.0,0.0,0.0,0.0,86060.0,2.0,NaN
1,2019,112002011.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,959.0,...,60000.0,1.0,3.0,6.0,0.0,0.0,0.0,8780.0,2.0,NaN
2,2019,112003011.0,NaN,NaN,NaN,NaN,25.0,0.0,0.0,78.0,...,74000.0,3.0,1.0,6.0,0.0,0.0,0.0,247670.0,2.0,NaN
3,2019,112010011.0,NaN,NaN,NaN,NaN,75.0,0.0,0.0,851.0,...,162600.0,1.0,3.0,6.0,450000.0,0.0,3800.0,3558000.0,2.0,NaN
4,2019,112012011.0,NaN,NaN,NaN,NaN,20.0,0.0,0.0,485.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,2041164.0,2.0,NaN


In [63]:
impute_nan0_hh = ['CUSTM_BENF1_D', 'CUSTM_BENF2_D','CUSTM_BENF3_D','CUSTM_BENF4_D','OTC_MED','HLT_SUP1','HLT_SUP2','MED_SUP','H_INC4','H_OOP','HEXP2','HEXP3','H_INC1','H_INC2','H_INC3','H_INC5','H_INC6','H_INC7','H_INC_TOT','H_INC_MON']
impute_nan6_hh = ['OTC_BUY1','OTC_BUY2','OTC_BUY3']
impute_9mean_hh = ['H_INC1','H_INC2','H_INC3','H_INC4','H_INC5','H_INC6','H_INC7','H_INC_TOT','H_INC_MON','MED_SUP','HEXP3']
encoding_hh = ['OTC_BUY1','OTC_BUY2','OTC_BUY3']

In [67]:
hh_pipeline = Pipeline([
    ('Imputer_1', ValueImputer(columns=impute_nan0_hh, missing_value=np.nan, fill_value=0)),
    ('Imputer_2', ValueImputer(columns=impute_nan6_hh, missing_value=np.nan, fill_value=6)),
    ('Imputer_3', SimpleImputer(strategy='median', missing_values=-9)),
    ('Column', ColumnsWithList(from_columns=encoding_hh, to_column='OTC_BUY')),
    ('Encoding', ManyHotEncoding(columns='OTC_BUY', prefix='OTC'))
])

In [68]:
hh_piped = hh_pipeline.fit_transform(hh)
hh_piped.head()

,OTC_1,OTC_2,OTC_3,OTC_4,OTC_5,OTC_6,D_YEAR,HHID,CUSTM_BENF1_D,CUSTM_BENF2_D,...,H_INC7,H_INC_TOT,H_INC_MON,OTC_MED,HLT_SUP1,HLT_SUP2,MED_SUP,H_OOP,HEXP2,HEXP3
0,1,0,1,0,0,1,2019.0,112001011.0,1.0,1.0,...,9.0,627.0,52.250000,61000.0,0.0,0.0,0.0,86060.0,2.0,0.0
1,1,0,1,0,0,1,2019.0,112002011.0,1.0,1.0,...,0.0,979.0,81.583333,60000.0,0.0,0.0,0.0,8780.0,2.0,0.0
2,1,0,1,0,0,1,2019.0,112003011.0,0.0,0.0,...,91.0,4497.0,374.750000,74000.0,0.0,0.0,0.0,247670.0,2.0,0.0
3,1,0,1,0,0,1,2019.0,112010011.0,0.0,0.0,...,0.0,2416.0,201.333333,162600.0,450000.0,0.0,3800.0,3558000.0,2.0,0.0
4,0,0,0,0,0,1,2019.0,112012011.0,0.0,0.0,...,232.0,1649.0,137.416667,0.0,0.0,0.0,0.0,2041164.0,2.0,0.0


# IND Table

## 만성질환 분류표

- 고혈압과 당뇨병 (HTN):
  - 고혈압, 당뇨병 (HTN, DM)

- 심뇌혈관 질환 (CVD):
  - 협심증, 심근경색증, 뇌출혈, 뇌경색 (AP, MI, CH, CI)

- 간질환 (LIV):
  - 만성간염(B형, C형), 알코올성 간질환, 간경화증(간경변증) (CLD, ALD, LC)

- 만성 하기도 질환(Lower Respiratory Infection, LRI):
  - 천식, 폐기종, 만성폐쇄성폐질환(COPD), 기관지확장증 (AST, PEM, COPD, BPE)

- 근골격계 질환 Musculoskeletal Disorders (MSD):
  - 무릎골관절염(무릎퇴행성관절염), 무릎 외 골관절염(퇴행성관절염), 류마티스 관절염, 어깨관절질환, 추간판(디스크) 질환, 기타 척추 질환 (OAK, OAE, RA, OAS, VD, VD_OLD, VDE, VDE_OLD)

- 갑상선 기능 장애 hyperthyroidism (HPT):
  - 갑상선 기능저하증, 갑상선 기능항진증 (HPOT, HPT)

In [12]:
# 질병 코드 그룹화
D_HTN = ["HTN", "DM"]
D_CVD = ["AP", "MI", "CH", "CI"]
D_LIV = ["CLD", "ALD", "LC"]
D_LRI = ["AST", "PEM", "COPD", "BPE"]
D_MSD_OLD = ["OAK", "OAE", "RA", "VD_OLD", "VDE_OLD"]
D_MSD = ["OAK", "OAE", "RA", "OAS", "VD", "VDE"]
D_MSD_ALL = ["OAK", "OAE", "RA", "OAS", "VD", "VDE", "VD_OLD", "VDE_OLD"]
D_HPT = ["HPOT", "HPT"]

# 열 이름으로 리스트화 (CD1 - 질병유무, CD2 - 진단시기)
D_HTN_CD1 = ["CD1_"+d for d in D_HTN]
D_CVD_CD1 = ["CD1_"+d for d in D_CVD]
D_LIV_CD1 = ["CD1_"+d for d in D_LIV]
D_LRI_CD1 = ["CD1_"+d for d in D_LRI]
D_MSD_OLD_CD1 = ["CD1_"+d for d in D_MSD_OLD]
D_MSD_CD1 = ["CD1_"+d for d in D_MSD]
D_HPT_CD1 = ["CD1_"+d for d in D_HPT]
D_OLD_CD1 = ["CD1_"+d for d in D_HTN + D_CVD + D_LIV + D_LRI + D_MSD_OLD + D_HPT]
D_21_CD1 = ["CD1_"+d for d in D_HTN + D_CVD + D_LIV + D_LRI + D_MSD + D_HPT]

## 만성질환을 그룹화 시킨 후 Many-Hot Encoding 형태로 변환 Transformer

In [13]:
class DiseaseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # 질병 분류 별 하나라도 해당하면 1, 아니면 0
        # 질병 분류 별 그룹화
        X["CD_HTN"] = X.apply(lambda x: 1 if any(x[col] == 1 for col in D_HTN_CD1) else 0, axis=1)
        X["CD_CVD"] = X.apply(lambda x: 1 if any(x[col] == 1 for col in D_CVD_CD1) else 0, axis=1)
        X["CD_LIV"] = X.apply(lambda x: 1 if any(x[col] == 1 for col in D_LIV_CD1) else 0, axis=1)
        X["CD_LRI"] = X.apply(lambda x: 1 if any(x[col] == 1 for col in D_LRI_CD1) else 0, axis=1)
        X["CD_MSD"] = X.apply(lambda x: 1 if any(x[col] == 1 for col in list(set(D_OLD_CD1 + D_21_CD1))) else 0, axis=1)
        X["CD_HPT"] = X.apply(lambda x: 1 if any(x[col] == 1 for col in D_HPT_CD1) else 0, axis=1)

        # 필요없는 행 탈락
        X.drop(X.iloc[:, 2:25], axis=1, inplace = True)

        return X

In [14]:
# 3개년 데이터의 질병 코드가 상이함. 따라서, 따로 불러온 후 그룹화 진행행
query = f"""
select PIDWON,2019 AS D_YEAR,CD1_HTN,CD1_DM,CD1_AP,CD1_MI,CD1_CH,CD1_CI,CD1_CLD,CD1_ALD,CD1_LC,CD1_AST,CD1_PEM,CD1_COPD,CD1_BPE,CD1_OAK,CD1_OAE,CD1_RA,CD1_VD_OLD,NULL AS CD1_VD,CD1_VDE_OLD,NULL AS CD1_VDE,CD1_HPOT,CD1_HPT,NULL AS CD1_OAS
from raw_a_ind
UNION ALL
select PIDWON,2020 AS D_YEAR,CD1_HTN,CD1_DM,CD1_AP,CD1_MI,CD1_CH,CD1_CI,CD1_CLD,CD1_ALD,CD1_LC,CD1_AST,CD1_PEM,CD1_COPD,CD1_BPE,CD1_OAK,CD1_OAE,CD1_RA,CD1_VD_OLD,NULL AS CD1_VD,CD1_VDE_OLD,NULL AS CD1_VDE,CD1_HPOT,CD1_HPT,NULL AS CD1_OAS
from raw_b_ind
UNION ALL
select PIDWON,2021 AS D_YEAR,CD1_HTN,CD1_DM,CD1_AP,CD1_MI,CD1_CH,CD1_CI,CD1_CLD,CD1_ALD,CD1_LC,CD1_AST,CD1_PEM,CD1_COPD,CD1_BPE,CD1_OAK,CD1_OAE,CD1_RA,NULL AS CD1_VD_OLD,CD1_VD,NULL AS CD1_VDE_OLD,CD1_VDE,CD1_HPOT,CD1_HPT,CD1_OAS
from raw_c_ind
"""

phi_disease = get_data_from_db(query)

In [15]:
disease_pipeline = Pipeline([
    ('disease', DiseaseTransformer())
])

In [16]:
ind_disease_piped = disease_pipeline.fit_transform(phi_disease)
ind_disease_piped.head()

,PIDWON,D_YEAR,CD_HTN,CD_CVD,CD_LIV,CD_LRI,CD_MSD,CD_HPT
0,11200101.0,2019,1,1,0,0,1,0
1,11200201.0,2019,0,0,0,0,1,0
2,11200202.0,2019,0,0,0,0,0,0
3,11200301.0,2019,0,0,0,0,0,0
4,11200302.0,2019,1,0,0,0,1,0


## 나머지 IND Table

In [17]:
columns_ind = """PIDWON, REGION1, SEX, MARR, EDU, EDU_STAT, HEALTH_INS, DISA_YN, DISA_TY, 
ECO1, ECO2, ECO3, ECO4, ECO5,ECO6,ECO7,ECO8,ECO9,ECO10,I_INC1_YN,I_INC1,I_INC2,
CARE2_1,CARE2_2,CARE2_3,CARE4_2,CARE5_1_1,CARE5_1_2,CARE5_1_3,
P1,REPLACE(P2, 8, 0), CASE WHEN WTMG = 4 THEN 0 ELSE 1 END AS WT_MG,coalesce(S4, (S5 / 30 * S6)) as S4,D1,
HT,WT,SE1,HS1,HS2_YN,HS3_YN,HS4_YN,HS5_YN,HS_MED_YN,HS6_1,HS7_1,HS8_1
"""

query_ind_2 = f"""
SELECT 2019 AS D_YEAR, {columns_ind}, 2019 - BIRTH_Y AS AGE, (P2_1 * 60 + P2_2) as P2_2
FROM raw_a_ind
UNION ALL
SELECT 2020 AS D_YEAR, {columns_ind}, 2020 - BIRTH_Y AS AGE, P2_2
FROM raw_b_ind
UNION ALL
SELECT 2021 AS D_YEAR, {columns_ind}, 2021 - BIRTH_Y AS AGE, P2_2
FROM raw_c_ind;
"""

ind = get_data_from_db(query_ind_2)
ind.head()

,D_YEAR,PIDWON,REGION1,SEX,MARR,EDU,EDU_STAT,HEALTH_INS,DISA_YN,DISA_TY,...,HS2_YN,HS3_YN,HS4_YN,HS5_YN,HS_MED_YN,HS6_1,HS7_1,HS8_1,AGE,P2_2
0,2019,11200101.0,26.0,2.0,3.0,1.0,NaN,5.0,2.0,NaN,...,2.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,81.0,30.0
1,2019,11200201.0,26.0,2.0,3.0,1.0,NaN,5.0,2.0,NaN,...,1.0,1.0,2.0,2.0,2.0,300.0,NaN,NaN,73.0,NaN
2,2019,11200202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,11200301.0,26.0,1.0,1.0,4.0,1.0,1.0,2.0,NaN,...,2.0,2.0,2.0,8.0,8.0,NaN,NaN,NaN,44.0,40.0
4,2019,11200302.0,26.0,2.0,1.0,4.0,1.0,2.0,2.0,NaN,...,2.0,2.0,2.0,8.0,8.0,45.0,NaN,NaN,41.0,20.0


In [18]:
impute_nan0_ind = ['DISA_TY', 'ECO1', 'ECO2', 'ECO3', 'ECO4', 'ECO5','ECO6','ECO7','ECO8','ECO9','ECO10','I_INC1_YN','I_INC1','I_INC2','CARE2_1','CARE2_2','CARE2_3','CARE4_2','CARE5_1_1','CARE5_1_2','CARE5_1_3','WT_MG','S4','D1','HS1','HS2_YN','HS3_YN','HS4_YN','HS6_1','HS7_1','HS8_1','P2_2']
impute_nan5_ind = ['EDU_STAT']
impute_nan8_ind = ['P2_2','HS5_YN','HS_MED_YN']
impute_nanmean_ind = ['HT','WT']
impute_90_ind = ['I_INC2']

In [19]:
ind_pipeline = Pipeline([
    ('Imputer_1', ValueImputer(columns=impute_nan0_ind, missing_value=np.nan, fill_value=0)),
    ('Imputer_2', ValueImputer(columns=impute_nan5_ind, missing_value=np.nan, fill_value=5)),
    ('Imputer_3', ValueImputer(columns=impute_nan8_ind, missing_value=np.nan, fill_value=8)),
    ('Imputer_4', ValueImputer(columns=impute_90_ind, missing_value=-9, fill_value=0)),
    ('Imputer_5', SimpleImputer(strategy='mean'))
    # 무엇을 원핫인코딩 할지? 먼저 떨구고 하자... 너무 많다..
])

In [20]:
ind_piped = ind_pipeline.fit_transform(ind)
ind_piped.head()

,D_YEAR,PIDWON,REGION1,SEX,MARR,EDU,EDU_STAT,HEALTH_INS,DISA_YN,DISA_TY,...,HS2_YN,HS3_YN,HS4_YN,HS5_YN,HS_MED_YN,HS6_1,HS7_1,HS8_1,AGE,P2_2
0,2019.0,11200101.0,26.000000,2.000000,3.000000,1.000000,5.0,5.000000,2.000000,0.0,...,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,81.000000,30.0
1,2019.0,11200201.0,26.000000,2.000000,3.000000,1.000000,5.0,5.000000,2.000000,0.0,...,1.0,1.0,2.0,2.0,2.0,300.0,0.0,0.0,73.000000,0.0
2,2019.0,11200202.0,36.109029,1.534087,2.417353,3.493523,5.0,2.291287,1.933285,0.0,...,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,49.568647,0.0
3,2019.0,11200301.0,26.000000,1.000000,1.000000,4.000000,1.0,1.000000,2.000000,0.0,...,2.0,2.0,2.0,8.0,8.0,0.0,0.0,0.0,44.000000,40.0
4,2019.0,11200302.0,26.000000,2.000000,1.000000,4.000000,1.0,2.000000,2.000000,0.0,...,2.0,2.0,2.0,8.0,8.0,45.0,0.0,0.0,41.000000,20.0


## 건강정보 이해능력 (C_IND) - 2021년 데이터에만 존재
- 변수 이름 변경

In [21]:
columns_c_ind = 'HHID,PIDWON,HLIT1, HLIT2,HLIT3,HLIT4,HLIT5,HLIT6,HLIT7,HLIT8,HLIT9,HLIT10,HLIT11,HLIT12,HLIT13,HLIT14,HLIT15,HLIT16,HLIT17,HLIT18_1,HLIT18_2,HLIT18_3'
query_c_ind = f"""SELECT 2021 AS D_YEAR, {columns_c_ind} FROM raw_c_ind"""

c_ind = get_data_from_db(query_c_ind)
c_ind.head()

,D_YEAR,HHID,PIDWON,HLIT1,HLIT2,HLIT3,HLIT4,HLIT5,HLIT6,HLIT7,...,HLIT11,HLIT12,HLIT13,HLIT14,HLIT15,HLIT16,HLIT17,HLIT18_1,HLIT18_2,HLIT18_3
0,2021,112001011.0,11200101.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,3.0,1.0,2.0,2.0,NaN,NaN,NaN
1,2021,112002011.0,11200201.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,...,3.0,2.0,2.0,3.0,2.0,3.0,2.0,NaN,NaN,NaN
2,2021,112002011.0,11200202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,112003011.0,11200301.0,2.0,2.0,3.0,3.0,3.0,3.0,4.0,...,3.0,3.0,3.0,4.0,4.0,4.0,2.0,NaN,NaN,NaN
4,2021,112003011.0,11200302.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,...,2.0,3.0,3.0,3.0,3.0,3.0,1.0,8.0,4.0,0.0


In [22]:
impute_nan6_c_ind = ['HLIT1','HLIT2','HLIT3','HLIT4','HLIT5','HLIT6','HLIT7','HLIT8','HLIT9','HLIT10','HLIT11','HLIT12','HLIT13','HLIT14','HLIT15','HLIT16','HLIT17']
impute_nan0_c_ind = ['HLIT18_1','HLIT18_2','HLIT18_3']
encoding_c_ind = ['HLIT18_1','HLIT18_2','HLIT18_3']

In [23]:
c_ind_pipeline = Pipeline([
    ('Imputer_1', ValueImputer(columns=impute_nan6_c_ind, missing_value=np.nan, fill_value=6)),
    ('Imputer_2', ValueImputer(columns=impute_nan0_c_ind, missing_value=np.nan, fill_value=0)),
    ('Column', ColumnsWithList(from_columns=encoding_c_ind, to_column='HLIT_MTD')),
    ('Encoding', ManyHotEncoding(columns='HLIT_MTD', prefix='HLIT_MTD'))
])

In [24]:
c_ind_piped = c_ind_pipeline.fit_transform(c_ind)
c_ind_piped

,HLIT_MTD_0,HLIT_MTD_1,HLIT_MTD_2,HLIT_MTD_3,HLIT_MTD_4,HLIT_MTD_5,HLIT_MTD_6,HLIT_MTD_7,HLIT_MTD_8,HLIT_MTD_9,...,HLIT8,HLIT9,HLIT10,HLIT11,HLIT12,HLIT13,HLIT14,HLIT15,HLIT16,HLIT17
0,1,0,0,0,0,0,0,0,0,0,...,1.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,2.0,2.0
1,1,0,0,0,0,0,0,0,0,0,...,2.0,3.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,2.0
2,1,0,0,0,0,0,0,0,0,0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
3,1,0,0,0,0,0,0,0,0,0,...,2.0,3.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,2.0
4,1,0,0,0,1,0,0,0,1,0,...,2.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13794,1,0,0,0,0,0,0,0,1,1,...,3.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,1.0
13795,0,0,0,0,1,0,0,0,1,1,...,2.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,1.0
13796,0,0,0,0,1,0,0,1,1,0,...,5.0,4.0,3.0,2.0,5.0,3.0,3.0,3.0,3.0,1.0
13797,0,1,0,0,0,0,0,0,1,1,...,3.0,3.0,3.0,3.0,2.0,3.0,4.0,3.0,3.0,1.0


## 최종 IND Table

In [25]:
tmp = pd.merge(ind_disease_piped, ind_piped, on=['PIDWON', 'D_YEAR'], how='outer')
ind_tmp = pd.merge(tmp, c_ind_piped, on=['PIDWON', 'D_YEAR'], how='outer')
ind_tmp.isnull().sum()[-30:] # 설문조사 부분이 NaN 값 많이 나옴 -> 모델에 쓸모 없을 것 같은데 그냥 버릴까?

AGE               0
P2_2              0
HLIT_MTD_0    31431
HLIT_MTD_1    31431
HLIT_MTD_2    31431
HLIT_MTD_3    31431
HLIT_MTD_4    31431
HLIT_MTD_5    31431
HLIT_MTD_6    31431
HLIT_MTD_7    31431
HLIT_MTD_8    31431
HLIT_MTD_9    31431
HHID          31431
HLIT1         31431
HLIT2         31431
HLIT3         31431
HLIT4         31431
HLIT5         31431
HLIT6         31431
HLIT7         31431
HLIT8         31431
HLIT9         31431
HLIT10        31431
HLIT11        31431
HLIT12        31431
HLIT13        31431
HLIT14        31431
HLIT15        31431
HLIT16        31431
HLIT17        31431
dtype: int64

# MS Table

In [26]:
columns_ms = 'HHID, PIDWON, MS1, DAYS, M_TYPE, MEXP1, MEXP2, MEXP3_1, MEXP3_1_1, MEXP3_1_2, MEXP3_2'

query_ms = f"""
SELECT 2019 AS D_YEAR, {columns_ms} FROM raw_a_ms
UNION ALL
SELECT 2020 AS D_YEAR, {columns_ms} FROM raw_b_ms
UNION ALL 
SELECT 2021 AS D_YEAR, {columns_ms} FROM raw_c_ms
"""

ms = get_data_from_db(query_ms)
ms.head()

,D_YEAR,HHID,PIDWON,MS1,DAYS,M_TYPE,MEXP1,MEXP2,MEXP3_1,MEXP3_1_1,MEXP3_1_2,MEXP3_2
0,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,-9.0,-9.0,-9.0
1,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,-9.0,-9.0,-9.0
2,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,-9.0,-9.0,-9.0
3,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,-9.0,-9.0,-9.0
4,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,-9.0,-9.0,-9.0


In [27]:
impute_90_ms = ['MEXP3_1','MEXP3_1_1','MEXP3_1_2','MEXP3_2']

In [28]:
ms_pipeline = Pipeline([
    ('Imputer', ValueImputer(columns=impute_90_ms, missing_value=-9, fill_value=0))
])

In [29]:
ms_piped = ms_pipeline.fit_transform(ms)
ms_piped

,D_YEAR,HHID,PIDWON,MS1,DAYS,M_TYPE,MEXP1,MEXP2,MEXP3_1,MEXP3_1_1,MEXP3_1_2,MEXP3_2
0,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
2,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
3,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
4,2019,112001011.0,11200101.0,3.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
776149,2021,400009011.0,40000901.0,3.0,1.0,4.0,1.0,2.0,0.0,0.0,0.0,0.0
776150,2021,400009011.0,40000901.0,3.0,1.0,1.0,1.0,1.0,50860.0,43300.0,7560.0,368412.0
776151,2021,400009011.0,40000901.0,3.0,1.0,1.0,1.0,1.0,11500.0,11500.0,0.0,104265.0
776152,2021,400009011.0,40000901.0,3.0,1.0,1.0,1.0,1.0,2100.0,2100.0,0.0,19940.0


# PHI Table
- transform 내용 정리리리리리리리

## PHI Table 에만 필요한 Transformer
- 내용내용

In [30]:
class PHITableTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # 단독 가구인 경우만 뽑기기
        X = X.loc[X["PHI_PID"].str.len() < 9, :]

        return X

## 3개년 병합해서 불러오기

In [31]:
columns_phi = "HHID,PHI_N,PHI_PID,PHI_PID1,PHI1,PHI2,PHI3,PHI4,PHI5,PHI6,PHR1,PHR2,PHR3"

query = f"""SELECT 2019 AS D_YEAR, {columns_phi} FROM raw_a_phi
            UNION ALL
            SELECT 2020 AS D_YEAR, {columns_phi} FROM raw_b_phi
            UNION ALL
            SELECT 2021 AS D_YEAR, {columns_phi} FROM raw_c_phi"""
phi = get_data_from_db(query)
phi.head()

,D_YEAR,HHID,PHI_N,PHI_PID,PHI_PID1,PHI1,PHI2,PHI3,PHI4,PHI5,PHI6,PHR1,PHR2,PHR3
0,2019,112003011.0,101.0,11200301,11200301.0,2004.0,-9.0,3.0,"1,2,3,8",1.0,120000.0,1.0,1.0,60000.0
1,2019,112003011.0,102.0,11200302,11200302.0,2004.0,-9.0,3.0,"1,2,3,8",1.0,118000.0,2.0,NaN,NaN
2,2019,112010011.0,101.0,11201001,11201001.0,2015.0,3.0,2.0,None,1.0,48710.0,2.0,NaN,NaN
3,2019,112010011.0,102.0,11201001,11201001.0,2000.0,7.0,1.0,"1,2",3.0,62800.0,2.0,NaN,NaN
4,2019,112010011.0,103.0,11201002,11201002.0,2012.0,2.0,3.0,"1,2,3,8",1.0,201717.0,2.0,NaN,NaN


## PHI Table 전처리 파이프라인

In [32]:
impute_nan2_phi = ['PHR2']
impute_nan0_phi = ['PHI4','PHR3']
impute_none0_phi = ['PHI4']
rename_map_phi = {'PHI_PID1' : 'PIDWON',
                  'PHI1' : 'PHI_Y',
                  'PHI2' : 'PHI_M',
                  'PHI3' : 'PHI_FORM',
                  'PHI4' : 'PHI_TYPE',
                  'PHI5' : 'PHI_PREMIUM_YN',
                  'PHI6' : 'PHI_PREMIUM',
                  'PHR1' : 'PHI_CLAIM_YN',
                  'PHR2' : 'PHI_BENEFIT_YN',
                  'PHR3' : 'PHI_BENEFIT'}
drop_columns_phi = ['PHI_PID']


In [33]:
phi_pipeline = Pipeline([
    ('Imputer_1', ValueImputer(columns=impute_nan2_phi, missing_value=np.nan, fill_value=2)),
    ('Imputer_2', ValueImputer(columns=impute_nan0_phi, missing_value=np.nan, fill_value=0)),
    ('Imputer_3', ValueImputer(columns=impute_none0_phi, missing_value=None, fill_value=0)),
    ('strip', StripTransformer(columns=['PHI_PID'])),
    ('phi_table', PHITableTransformer()),
    ('rename_drop', RenameDropColumnsTransformer(rename_map=rename_map_phi, drop_columns=drop_columns_phi))
])

In [34]:
phi_piped = phi_pipeline.fit_transform(phi)
phi_piped

,D_YEAR,HHID,PHI_N,PIDWON,PHI_Y,PHI_M,PHI_FORM,PHI_TYPE,PHI_PREMIUM_YN,PHI_PREMIUM,PHI_CLAIM_YN,PHI_BENEFIT_YN,PHI_BENEFIT
0,2019,112003011.0,101.0,11200301.0,2004.0,-9.0,3.0,"1,2,3,8",1.0,120000.0,1.0,1.0,60000.0
1,2019,112003011.0,102.0,11200302.0,2004.0,-9.0,3.0,"1,2,3,8",1.0,118000.0,2.0,2.0,0
2,2019,112010011.0,101.0,11201001.0,2015.0,3.0,2.0,0,1.0,48710.0,2.0,2.0,0
3,2019,112010011.0,102.0,11201001.0,2000.0,7.0,1.0,"1,2",3.0,62800.0,2.0,2.0,0
4,2019,112010011.0,103.0,11201002.0,2012.0,2.0,3.0,"1,2,3,8",1.0,201717.0,2.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64270,2021,400007011.0,201.0,40000703.0,-9.0,-9.0,1.0,1,1.0,-9.0,2.0,2.0,0
64271,2021,400007011.0,202.0,40000704.0,-9.0,-9.0,1.0,1,1.0,-9.0,2.0,2.0,0
64272,2021,400008011.0,102.0,40000802.0,2014.0,12.0,3.0,"1,3",1.0,71740.0,1.0,1.0,506871.0
64273,2021,400008011.0,401.0,40000802.0,2018.0,5.0,1.0,2,1.0,21550.0,2.0,2.0,0


# PHI 기준으로 합치기

In [35]:
phi_piped

,D_YEAR,HHID,PHI_N,PIDWON,PHI_Y,PHI_M,PHI_FORM,PHI_TYPE,PHI_PREMIUM_YN,PHI_PREMIUM,PHI_CLAIM_YN,PHI_BENEFIT_YN,PHI_BENEFIT
0,2019,112003011.0,101.0,11200301.0,2004.0,-9.0,3.0,"1,2,3,8",1.0,120000.0,1.0,1.0,60000.0
1,2019,112003011.0,102.0,11200302.0,2004.0,-9.0,3.0,"1,2,3,8",1.0,118000.0,2.0,2.0,0
2,2019,112010011.0,101.0,11201001.0,2015.0,3.0,2.0,0,1.0,48710.0,2.0,2.0,0
3,2019,112010011.0,102.0,11201001.0,2000.0,7.0,1.0,"1,2",3.0,62800.0,2.0,2.0,0
4,2019,112010011.0,103.0,11201002.0,2012.0,2.0,3.0,"1,2,3,8",1.0,201717.0,2.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64270,2021,400007011.0,201.0,40000703.0,-9.0,-9.0,1.0,1,1.0,-9.0,2.0,2.0,0
64271,2021,400007011.0,202.0,40000704.0,-9.0,-9.0,1.0,1,1.0,-9.0,2.0,2.0,0
64272,2021,400008011.0,102.0,40000802.0,2014.0,12.0,3.0,"1,3",1.0,71740.0,1.0,1.0,506871.0
64273,2021,400008011.0,401.0,40000802.0,2018.0,5.0,1.0,2,1.0,21550.0,2.0,2.0,0


In [36]:
phi_ind = pd.merge(phi_piped, tmp, on=['PIDWON','D_YEAR'], how='left')

In [37]:
# 가구데이터 추가
except_ms = pd.merge(phi_ind, hh_piped, on=["HHID", "D_YEAR"], how='left')
except_ms

,D_YEAR,HHID,PHI_N,PIDWON,PHI_Y,PHI_M,PHI_FORM,PHI_TYPE,PHI_PREMIUM_YN,PHI_PREMIUM,...,H_INC7,H_INC_TOT,H_INC_MON,OTC_MED,HLT_SUP1,HLT_SUP2,MED_SUP,H_OOP,HEXP2,HEXP3
0,2019,112003011.0,101.0,11200301.0,2004.0,-9.0,3.0,"1,2,3,8",1.0,120000.0,...,91.0,4497.0,374.750000,74000.0,0.0,0.0,0.0,247670.0,2.0,0.0
1,2019,112003011.0,102.0,11200302.0,2004.0,-9.0,3.0,"1,2,3,8",1.0,118000.0,...,91.0,4497.0,374.750000,74000.0,0.0,0.0,0.0,247670.0,2.0,0.0
2,2019,112010011.0,101.0,11201001.0,2015.0,3.0,2.0,0,1.0,48710.0,...,0.0,2416.0,201.333333,162600.0,450000.0,0.0,3800.0,3558000.0,2.0,0.0
3,2019,112010011.0,102.0,11201001.0,2000.0,7.0,1.0,"1,2",3.0,62800.0,...,0.0,2416.0,201.333333,162600.0,450000.0,0.0,3800.0,3558000.0,2.0,0.0
4,2019,112010011.0,103.0,11201002.0,2012.0,2.0,3.0,"1,2,3,8",1.0,201717.0,...,0.0,2416.0,201.333333,162600.0,450000.0,0.0,3800.0,3558000.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63686,2021,400007011.0,201.0,40000703.0,-9.0,-9.0,1.0,1,1.0,-9.0,...,0.0,6080.0,506.666667,400000.0,200000.0,0.0,0.0,4422620.0,2.0,0.0
63687,2021,400007011.0,202.0,40000704.0,-9.0,-9.0,1.0,1,1.0,-9.0,...,0.0,6080.0,506.666667,400000.0,200000.0,0.0,0.0,4422620.0,2.0,0.0
63688,2021,400008011.0,102.0,40000802.0,2014.0,12.0,3.0,"1,3",1.0,71740.0,...,51.0,7746.0,645.500000,143000.0,500000.0,300000.0,0.0,3600770.0,2.0,0.0
63689,2021,400008011.0,401.0,40000802.0,2018.0,5.0,1.0,2,1.0,21550.0,...,51.0,7746.0,645.500000,143000.0,500000.0,300000.0,0.0,3600770.0,2.0,0.0
